<a href="https://colab.research.google.com/github/ayseakbaba/CS-Repos/blob/main/Recommender_System_Using_With_Cornac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cornac

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 61.5 MB/s eta 0:00:00


In [74]:
import cornac as cn
from sklearn.model_selection import train_test_split
from cornac.utils import cache
import pandas as pd
import numpy as np

In [4]:
data = cn.datasets.movielens.load_feedback()

Data from http://files.grouplens.org/datasets/movielens/ml-100k/u.data
will be cached into /root/.cornac/ml-100k/u.data


0.00B [00:00, ?B/s]

File cached!


In [10]:
ratio_split = cn.eval_methods.RatioSplit(
    data=data, test_size=0.2, rating_threshold=4.0, verbose=True
)

rating_threshold = 4.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1653
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1653
Number of ratings = 19970
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1653


In [8]:
svd = cn.models.SVD(
    k=10, max_iter=5, learning_rate=0.01, lambda_reg=0.02, verbose=True
)

In [59]:
nmf = cn.models.NMF(
    k=10, max_iter=5, learning_rate=0.01, lambda_reg=0.02, verbose=True
)

In [13]:
mae = cn.metrics.MAE()
rmse = cn.metrics.RMSE()
precision = cn.metrics.Precision(5)
recall = cn.metrics.Recall(5)

In [60]:
cn.Experiment(eval_method=ratio_split, models=[svd, nmf], metrics=[mae, rmse, precision, recall]).run()


[SVD] Training started!


/usr/local/lib/python3.10/dist-packages/cornac/models/recommender.py:322: UserWarning: Model is already fitted. Re-fitting will overwrite the previous model.
  warnings.warn(


  0%|          | 0/5 [00:00<?, ?it/s]

Optimization finished!

[SVD] Evaluation started!


Rating:   0%|          | 0/19970 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

Optimization finished!

[NMF] Evaluation started!


Rating:   0%|          | 0/19970 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
    |    MAE |   RMSE | Precision@5 | Recall@5 | Train (s) | Test (s)
--- + ------ + ------ + ----------- + -------- + --------- + --------
SVD | 0.7627 | 0.9207 |      0.0595 |   0.0245 |    0.0602 |   1.3672
NMF | 0.9583 | 1.2082 |      0.0009 |   0.0003 |    0.0597 |   1.1275



In [52]:
movie_df = pd.read_csv(
  cache("http://files.grouplens.org/datasets/movielens/ml-100k/u.item"),
  sep="|", encoding="ISO-8859-1",
  names=["ItemID", "Title", "Release Date", "Video Release Date", "IMDb URL",
         "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy",
         "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
         "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
).set_index("ItemID").drop(columns=["Video Release Date", "IMDb URL", "unknown"])

In [79]:
def get_recommendations(algo, user_id, movie_df, n=5):
  predicted_movie_ids = algo.recommend(user_id, n)
  movieList = []
  for predict in predicted_movie_ids:
    predict = int(predict)
    score = algo.rate(int(user_id), predict)
    movieList.append((movie_df.iloc[predict].iloc[0], np.round(score)))
  return movieList

In [80]:
print("SVD Algoritmasına Göre Öneriler:")
print(get_recommendations(svd, '1', movie_df))


print("\nNMF Algoritmasına Göre Öneriler:")
print(get_recommendations(nmf, '1', movie_df))

SVD Algoritmasına Göre Öneriler:
[('Jack (1996)', 3.0), ('Maltese Falcon, The (1941)', 2.0), ('Cinema Paradiso (1988)', 2.0), ('Supercop (1992)', 2.0), ('Haunted World of Edward D. Wood Jr., The (1995)', 3.0)]

NMF Algoritmasına Göre Öneriler:
[('Object of My Affection, The (1998)', 3.0), ('Secret Adventures of Tom Thumb, The (1993)', 2.0), ('Scarlet Letter, The (1926)', 3.0), ('Cosi (1996)', 4.0), ('Eighth Day, The (1996)', 3.0)]
